In [1]:
import pandas as pd
import numpy as np

from tqdm import tqdm #시간 측정용

# ---------- 한글 --------- #
import MeCab
mec = MeCab.Tagger()

import pickle

import datetime
from datetime import timedelta

from collections import Counter

from keras.preprocessing.text import Tokenizer

pd.set_option("display.max_rows", 500)

Using TensorFlow backend.


In [2]:
# ------------------------------- Data load ------------------------------- #
beh = pd.read_csv("./Data/제6회 L.POINT Big Data Competition-분석용데이터-01.온라인 행동 정보.csv")
tra = pd.read_csv("./Data/제6회 L.POINT Big Data Competition-분석용데이터-02.거래 정보.csv")
cus = pd.read_csv("./Data/제6회 L.POINT Big Data Competition-분석용데이터-03.고객 Demographic 정보.csv")
cla = pd.read_csv("./Data/제6회 L.POINT Big Data Competition-분석용데이터-04.상품분류 정보.csv")

E:\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3057: DtypeWarning: Columns (9) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [21]:
# pd_c 추천 함수
pd_cArr = np.load("./Keyword in Clac_nm123/pd_cArr.npy")
def Recommend_PD_C(pdc,categorylen):
    if len(Counter(pd_cArr[pdc-1])) <= categorylen:
        categorylen = len(Counter(pd_cArr[pdc-1]))
        if categorylen == 0:
            print("추천 할 수 있는 데이터가 없습니다.")
            return -1
    return [key for key, _ in Counter(pd_cArr[pdc-1]).most_common(categorylen)]

In [5]:
# ----------------------------------------------- PD_C 배열 생성 ----------------------------------------------- #
# Trans ID
tid = tra["trans_id"].drop_duplicates().values
tra = tra.sort_values(["trans_id","trans_seq","clnt_id"])
tra = tra[tra["pd_c"]!='unknown']

# 배열  0 ~ 1666
pd_cArr = []
for i in range(1667):
    pd_cArr.append([])
    
for i in tqdm(tid):
    temp_pd_cArr = tra[tra["trans_id"]==i]["pd_c"].astype(int).values.tolist()
    for i in temp_pd_cArr:
        # 자기자신 빼고 입력
        temp_pd_cArr.remove(i)
        pd_cArr[i-1] += cla[cla["pd_c"].isin(temp_pd_cArr)]["clac_nm3"].drop_duplicates().values.tolist()
        temp_pd_cArr.append(i)
# ----------------------------------------------- PD_C 배열 생성 ----------------------------------------------- #

100%|█████████████████████████████████████████████████████████████████████████| 116440/116440 [12:01<00:00, 161.48it/s]


In [6]:
np.save("./Keyword in Clac_nm123/pd_cArr",pd_cArr)